In [ ]:
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Instantiate GitHub Toolkit
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())
tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

agent_executor = create_react_agent(llm, tools)

def ai(query: str):
    # System prompt allows for broad range of instructions. But what does user input look like?
    events = agent_executor.stream(
        {"messages": [("system",
                       "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                       "All output must be in HTML format and will be displayed to an end user. "
                       "You need to start with doctype and html tags and provide the entire page"),
                      ("user", query)]},
        stream_mode="values",
    )
    data = ""
    for event in events:
        event["messages"][-1].pretty_print()
        data = event["messages"][-1]

    return data.content


In [ ]:
from pathlib import Path

from dotenv import load_dotenv
from flask import Flask


# Load environment variables from .env
load_dotenv()

app = Flask(__name__)

@app.route('/', methods=['GET'])
def get_data():
    # Wow where is the logic?
    return ai(Path("my_app1.txt").read_text())


## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~

In [4]:
app.run(debug=False, port=5001, use_reloader=False)

================================ Human Message =================================

Find the most prolific author of github issues, and I want to see all of their issues and comments

I will need to see ONLY the author, title, and comments. HTML elements can be styled using twitter bootstrap css.

I want to see comments only by the author of the issue.

I want to see the comments in an accordion style dropdown.

Give me a link to each issue. My repo is breba-apps/TempRepo. Make sure the link is not on the accordion itself, but a completely separate link
================================== Ai Message ==================================
Tool Calls:
  get_issues (call_S9YIhJLmjVEej9wn3HkChuqO)
 Call ID: call_S9YIhJLmjVEej9wn3HkChuqO
  Args:
================================= Tool Message =================================
Name: get_issues

Found 2 issues:
[{'title': 'This is my second issue', 'number': 2, 'opened_by': 'yasonk'}, {'title': 'First issue here you go', 'number': 1, 'opened_by': 'ya

127.0.0.1 - - [12/Mar/2025 12:28:48] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues by Most Prolific Author</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">Issues by Most Prolific Author: yasonk</h1>
        
        <div class="card mb-3">
            <div class="card-header">
                <h5 class="mb-0">
                    <a href="https://github.com/breba-apps/TempRepo/issues/2" target="_blank">This is my second issue</a>
                </h5>
            </div>
            <div class="card-body">
                <div id="accordion2">
                    <div class="card">
                        <div class="card-header" id="headingTwo">
                            <h5 class="

127.0.0.1 - - [12/Mar/2025 12:32:16] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Available Tools</title>
</head>
<body>
    <h1>Available Tools</h1>
    <p>Here is a list of tools that I can use to interact with a GitHub repository:</p>
    <ul>
        <li><strong>get_issues:</strong> Fetches a list of the repository's issues.</li>
        <li><strong>get_issue:</strong> Fetches the title, body, and comment thread of a specific issue.</li>
        <li><strong>comment_on_issue:</strong> Allows commenting on a GitHub issue.</li>
        <li><strong>list_open_pull_requests:</strong> Fetches a list of the repository's open Pull Requests (PRs).</li>
        <li><strong>get_pull_request:</strong> Fetches the title, body, comment thread, and commit history of a specific Pull Request.</li>
        <li><strong>list_pull_request_files:</strong